In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
%matplotlib inline
from __future__ import division
from __future__ import print_function
from scipy.optimize import curve_fit
import os

In [2]:
datadirectory = 'data_membrane_meas'
def makename(filename):
    return os.path.join(datadirectory,filename)

In [3]:
class membraneThickness(object):
    'Data and fit parameters for membrane thickness measurements as a function of layer exposure time'
    
    def __init__(self,data=None,filepathname=''):
        if data is not None:
            self.data = data
            self.rawmeasdata_pix = None
        elif filepathname != '':
            self.rawmeasdata_pix = np.loadtxt(filepathname, delimiter=',', skiprows=1)
            layerexptimes = self.rawmeasdata_pix[0,:]
            self.rawmeasdata_um, tempave, tempstd = self.calc_ave_stdev_membrane_thickness_um(
                self.rawmeasdata_pix[1:,:])
            self.data = np.array([layerexptimes,tempave,tempstd]).T
        else:
            print('ERROR: NO DATA OR FILENAME SPECIFIED')
            return
        self.fitparams, self.covmatrix = curve_fit(self.membthickfunc, self.data[:,0], self.data[:,1])
        self.std_devs = np.sqrt(self.covmatrix.diagonal())
    
    def membthickfunc(self,t,a,b):
        return a*np.log(t/b)
    
    def calc_ave_stdev_membrane_thickness_um(self,rawdata):
        # Convert from pixels to microns (assumes pixel measurements are in x-direction)
        rawdata[rawdata == 0] = np.NaN
        rawdata = rawdata * 1.8244
        columnaverages = np.nanmean(rawdata, axis=0)
        columnstdev = np.nanstd(rawdata, axis=0)
        return rawdata, columnaverages, columnstdev

In [4]:
a = membraneThickness(filepathname=makename('150415_10irga_01btex.csv'))
print(a.data)
print(a.fitparams)

[[  2.00000000e+00   6.83846541e+02   2.33027140e+01]
 [  1.00000000e+00   3.92854741e+02   8.72831309e+00]
 [  5.00000000e-01   2.15281633e+02   1.65899401e+01]
 [  2.50000000e-01   4.92653678e+01   1.25350024e+01]]
[  3.00270518e+02   2.31477284e-01]


/Users/nordin/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:21: RuntimeWarning: invalid value encountered in log


In [5]:
data_0330 = np.array([[   0.3,    54.5,     9.49],
 [   0.35,  105.7,    15.94],
 [   0.4,   127.62,   13.19],
 [   0.45,  185.95,   15.6 ],
 [   0.5,   197.56,   16.36]])
a = membraneThickness(data_0330)
print(a.data)
print(a.fitparams)

[[   0.3    54.5     9.49]
 [   0.35  105.7    15.94]
 [   0.4   127.62   13.19]
 [   0.45  185.95   15.6 ]
 [   0.5   197.56   16.36]]
[  2.88233065e+02   2.47048025e-01]


/Users/nordin/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:21: RuntimeWarning: invalid value encountered in log
